# EA1 — Siniestralidad Vial en Medellín

**Objetivo:** crear una base local en **SQLite** desde el dataset *Incidentes viales* y exportar un CSV desde la base para evidenciar el flujo **dataset → SQLite → CSV**.

## 1) Preparación e inspección rápida

In [2]:
import os, sqlite3, csv
import pandas as pd
from pathlib import Path

PROJECT_ROOT = '/mnt/data/proyecto_integrado_ea1'
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
DB_DIR = os.path.join(PROJECT_ROOT, 'db')
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(DB_DIR, exist_ok=True)

NB_DIR = Path.cwd()
PROJECT_ROOT = NB_DIR.parent if NB_DIR.name.lower() == "notebooks" else NB_DIR

DATA_DIR = PROJECT_ROOT / "data"
DB_DIR   = PROJECT_ROOT / "db"
DB_DIR.mkdir(parents=True, exist_ok=True)

csv_candidates = list(DATA_DIR.glob("incidentes_viales.csv"))
assert csv_candidates, f"No hay archivos .csv en {DATA_DIR}. Copia el dataset allí."
CSV_PATH = csv_candidates[0]

print("Proyecto:", PROJECT_ROOT)
print("CSV usado:", CSV_PATH)
print("DB dir   :", DB_DIR)

preview = pd.read_csv(CSV_PATH, nrows=15, low_memory=False)
print('Columnas originales:', list(preview.columns))
preview.head(15)


Proyecto: c:\Dev\proyecto_integrado_ea1
CSV usado: c:\Dev\proyecto_integrado_ea1\data\incidentes_viales.csv
DB dir   : c:\Dev\proyecto_integrado_ea1\db
Columnas originales: ['AÑO', 'CBML', 'CLASE_ACCIDENTE', 'DIRECCION', 'DIRECCION ENCASILLADA', 'DISEÑO', 'EXPEDIENTE', 'FECHA_ACCIDENTE', 'FECHA_ACCIDENTES', 'GRAVEDAD_ACCIDENTE', 'MES', 'NRO_RADICADO', 'NUMCOMUNA', 'BARRIO', 'COMUNA', 'LOCATION', 'X', 'Y']


,AÑO,CBML,CLASE_ACCIDENTE,DIRECCION,DIRECCION ENCASILLADA,DISEÑO,EXPEDIENTE,FECHA_ACCIDENTE,FECHA_ACCIDENTES,GRAVEDAD_ACCIDENTE,MES,NRO_RADICADO,NUMCOMUNA,BARRIO,COMUNA,LOCATION,X,Y
0,2015,1013,Caida Ocupante,CR 46 CL 43,CR 046 043 000 00000,Tramo de via,A000259731,21/10/2015 05:58:00,2015-10-21T10:58:00.000Z,Con heridos,10,1508668,10,Barrio Colón,La Candelaria,"[-75.5688011014, 6.24312304123]",834949.69,1182357.34
1,2015,1020,Choque,CR 43 A CL 29,CR 043 A 029 000 00000,Tramo de via,A000261725,05/11/2015 03:00:00,2015-11-05T08:00:00.000Z,Solo daños,11,1510621,10,San Diego,La Candelaria,"[-75.5693883283, 6.22870030622]",834880.17,1180762.02
2,2015,205,Otro,CR 46 CL 37,CR 046 037 000 00000,Tramo de via,A000259739,21/10/2015 07:40:00,2015-10-21T12:40:00.000Z,Con heridos,10,1508691,2,La Francia,Santa Cruz,"[-75.5503911403, 6.29869502502]",837004.94,1188499.17
3,2015,101,Atropello,CR 34 CL 112,CR 034 112 000 00000,Tramo de via,A000259752,21/10/2015 12:00:00,2015-10-21T17:00:00.000Z,Con heridos,10,1508694,1,Santo Domingo Savio No. 1,Popular,"[-75.5442195069, 6.30019383306]",837688.53,1188663.05
4,2015,1504,Choque,CR 50 CL 14,CR 050 014 000 00000,Tramo de via,A000259949,03/11/2015 06:16:00,2015-11-03T11:16:00.000Z,Solo daños,11,1510634,15,Santa Fé,Guayabal,"[-75.578180998, 6.21699243701]",833903.11,1179469.60
5,2015,1417,Choque,TV 7 A CL 30,TV 007 A 030 000 00000,Tramo de via,A000261734,05/11/2015 04:30:00,2015-11-05T09:30:00.000Z,Solo daños,11,1510646,14,La Florida,El Poblado,"[-75.561118009, 6.2047775959]",835788.30,1178113.02
6,2015,1105,Choque,CR 63 C CL 42 C,CR 063 C 042 C 000 00000,Tramo de via,A000259808,18/10/2015 04:40:00,2015-10-18T09:40:00.000Z,Solo daños,10,1508740,11,Los Conquistadores,Laureles Estadio,"[-75.5810144962, 6.24576066323]",833598.49,1182652.97
7,2015,1009,Choque,CR 57 CL 44,CR 057 044 000 00000,Tramo de via,A000261730,04/11/2015 04:46:00,2015-11-04T09:46:00.000Z,Solo daños,11,1510650,10,La Alpujarra,La Candelaria,"[-75.5752597216, 6.24685341912]",834235.89,1182772.04
8,2015,1018,Choque,CL 57 CR 46,CL 057 046 000 00000,Tramo de via,A000259726,21/10/2015 07:00:00,2015-10-21T12:00:00.000Z,Con heridos,10,1508766,10,Villa Nueva,La Candelaria,"[-75.5624178534, 6.25341454089]",835659.53,1183493.83
9,2015,705,Choque,CR 80 CL 66,CR 080 066 000 00000,Tramo de via,A000261644,05/11/2015 07:00:00,2015-11-05T12:00:00.000Z,Con heridos,11,1510724,7,Facultad de Minas U. Nacional,Robledo,"[-75.5931430393, 6.27330476643]",832264.64,1185703.90


## 2) Normalización y funciones auxiliares

In [3]:
import pandas as pd

def norm_col(c: str) -> str:
    c = c.strip().lower().replace(' ', '_')
    c = (c.replace('á','a').replace('é','e').replace('í','i')
           .replace('ó','o').replace('ú','u').replace('ñ','n'))
    while '__' in c:
        c = c.replace('__','_')
    return c

def parse_fecha(df: pd.DataFrame):
    col = None
    for candidate in ['fecha_accidentes', 'fecha_accidente']:
        if candidate in df.columns:
            col = candidate
            break
    if col:
        dt = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)
        fechas = dt.dt.strftime('%Y-%m-%d')
        horas  = dt.dt.strftime('%H:%M:%S')
        return fechas, horas
    if 'ano' in df.columns and 'mes' in df.columns:
        dt = pd.to_datetime(df['ano'].astype(str) + '-' + df['mes'].astype(str) + '-01', errors='coerce')
        return dt.dt.strftime('%Y-%m-%d'), pd.Series([None]*len(df), dtype='object')
    return pd.Series([None]*len(df), dtype='object'), pd.Series([None]*len(df), dtype='object')


## 3) Creación de la base y carga por lotes (chunks)

In [4]:
import sqlite3

DB_PATH = os.path.join(DB_DIR, 'proyecto.db')
con = sqlite3.connect(DB_PATH)
cur = con.cursor()

# Tablas destino
cur.execute('DROP TABLE IF EXISTS incidentes;')
cur.execute('DROP TABLE IF EXISTS incidentes_raw;')
con.commit()

chunksize = 50000
total = 0
raw_initialized = False

for chunk in pd.read_csv(CSV_PATH, low_memory=False, chunksize=chunksize):
    # Normalizar columnas
    chunk.columns = [norm_col(c) for c in chunk.columns]
    # Crear RAW la primera vez
    if not raw_initialized:
        cols_sql = ', '.join([f'"{c}" TEXT' for c in chunk.columns])
        cur.execute(f'CREATE TABLE incidentes_raw ({cols_sql});')
        con.commit()
        raw_initialized = True

    # Insertar en RAW
    placeholders = ', '.join(['?']*len(chunk.columns))
    col_list = ', '.join([f'"{c}"' for c in chunk.columns])
    cur.executemany(
        f'INSERT INTO incidentes_raw ({col_list}) VALUES ({placeholders});',
        chunk.astype(str).where(pd.notna(chunk), None).values.tolist()
    )
    con.commit()

    # Procesado mínimo
    fechas, horas = parse_fecha(chunk)
    proc = pd.DataFrame({
        'fecha': fechas,
        'hora': horas,
        'anio': chunk['ano'] if 'ano' in chunk.columns else None,
        'mes': chunk['mes'] if 'mes' in chunk.columns else None,
        'comuna_codigo': chunk['numcomuna'] if 'numcomuna' in chunk.columns else None,
        'comuna': chunk['comuna'] if 'comuna' in chunk.columns else None,
        'barrio': chunk['barrio'] if 'barrio' in chunk.columns else None,
        'clase_accidente': chunk['clase_accidente'] if 'clase_accidente' in chunk.columns else None,
        'gravedad': chunk['gravedad_accidente'] if 'gravedad_accidente' in chunk.columns else None,
        'diseno': chunk['diseno'] if 'diseno' in chunk.columns else None,
        'direccion': chunk['direccion'] if 'direccion' in chunk.columns else None,
        'direccion_encasillada': chunk['direccion_encasillada'] if 'direccion_encasillada' in chunk.columns else None,
        'x_magna': chunk['x'] if 'x' in chunk.columns else None,
        'y_magna': chunk['y'] if 'y' in chunk.columns else None,
        'cbml': chunk['cbml'] if 'cbml' in chunk.columns else None,
        'expediente': chunk['expediente'] if 'expediente' in chunk.columns else None,
        'nro_radicado': chunk['nro_radicado'] if 'nro_radicado' in chunk.columns else None,
        'location': chunk['location'] if 'location' in chunk.columns else None,
    })

    # Crear tabla destino si no existe
    cur.execute('CREATE TABLE IF NOT EXISTS incidentes (id INTEGER PRIMARY KEY AUTOINCREMENT, fecha TEXT, hora TEXT, anio INTEGER, mes INTEGER, comuna_codigo INTEGER, comuna TEXT, barrio TEXT, clase_accidente TEXT, gravedad TEXT, diseno TEXT, direccion TEXT, direccion_encasillada TEXT, x_magna REAL, y_magna REAL, cbml TEXT, expediente TEXT, nro_radicado TEXT, location TEXT);')
    con.commit()

    insert_cols = ['fecha','hora','anio','mes','comuna_codigo','comuna','barrio',
                   'clase_accidente','gravedad','diseno','direccion','direccion_encasillada',
                   'x_magna','y_magna','cbml','expediente','nro_radicado','location']
    ph = ', '.join(['?']*len(insert_cols))
    cur.executemany(
        f'INSERT INTO incidentes ({", ".join(insert_cols)}) VALUES ({ph});',
        proc[insert_cols].where(pd.notna(proc), None).values.tolist()
    )
    con.commit()

    total += len(chunk)
    print(f'+{len(chunk)} filas (acum: {total})')

print('Carga finalizada. Total filas procesadas:', total)


C:\Users\David Sanchez\AppData\Local\Temp\ipykernel_3564\2764285548.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)


+50000 filas (acum: 50000)


C:\Users\David Sanchez\AppData\Local\Temp\ipykernel_3564\2764285548.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)


+50000 filas (acum: 100000)


C:\Users\David Sanchez\AppData\Local\Temp\ipykernel_3564\2764285548.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)


...

+50000 filas (acum: 200000)


C:\Users\David Sanchez\AppData\Local\Temp\ipykernel_3564\2764285548.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)


+50000 filas (acum: 250000)


C:\Users\David Sanchez\AppData\Local\Temp\ipykernel_3564\2764285548.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)


+20765 filas (acum: 270765)
Carga finalizada. Total filas procesadas: 270765


## 4) Índices y verificación

In [5]:
cur.execute('CREATE INDEX IF NOT EXISTS idx_incidentes_fecha ON incidentes(fecha);')
cur.execute('CREATE INDEX IF NOT EXISTS idx_incidentes_comuna ON incidentes(comuna);')
cur.execute('CREATE INDEX IF NOT EXISTS idx_incidentes_clase ON incidentes(clase_accidente);')
con.commit()

n_raw = cur.execute('SELECT COUNT(*) FROM incidentes_raw;').fetchone()[0]
n_inc = cur.execute('SELECT COUNT(*) FROM incidentes;').fetchone()[0]
print('Registros en incidentes_raw:', n_raw)
print('Registros en incidentes     :', n_inc)

import pandas as pd
pd.read_sql_query('SELECT * FROM incidentes ORDER BY fecha, hora LIMIT 10;', con)


Registros en incidentes_raw: 270765
Registros en incidentes     : 270765


,id,fecha,hora,anio,mes,comuna_codigo,comuna,barrio,clase_accidente,gravedad,diseno,direccion,direccion_encasillada,x_magna,y_magna,cbml,expediente,nro_radicado,location
0,136709,2014-07-04,10:40:00,2014,7,7,Robledo,Facultad de Minas U. Nacional,Choque,Solo daños,Tramo de via,CL 65 CR 78,CL 065 078 000 00000,832352.33,1185661.63,0705,A000091015,1447988,"[-75.5923497685, 6.27292496093]"
1,221549,2014-07-04,11:50:00,2014,7,10,La Candelaria,Guayaquil,Choque,Solo daños,Tramo de via,CL 44 CR 51,CL 044 051 000 00000,834728.51,1182534.62,1007,A000091006,1447987,"[-75.570803628, 6.24471990473]"
2,136402,2014-07-04,12:10:00,2014,7,16,Belén,San Bernardo,Choque,Solo daños,Tramo de via,CR 76 CL 17 A,CR 076 017 A 000 00000,831746.23,1180109.45,1605,A000091492,1448074,"[-75.5976802402, 6.22272063984]"
3,177702,2014-07-04,13:15:00,2014,7,10,La Candelaria,Calle Nueva,Choque,Solo daños,Glorieta,CR 55 CL 37,CR 055 037 000 00000,834206.83,1182010.64,1011,A000090986,1447986,"[-75.5755026696, 6.2399699888]"
4,233263,2014-07-04,13:20:00,2014,7,5,Castilla,Terminal de Transporte,Choque,Con muertos,None,CR 64 CL 75B,CR 064 075 B 000 00000,834756.85,1185787.55,0516,A000090994,32295,"[-75.5706311344, 6.27412557275]"
5,45765,2014-07-04,14:40:00,2014,7,16,Belén,El Rincón,Choque,Solo daños,Tramo de via,CR 81 A CL 1,CR 081 A 001 000 00000,830999.92,1178767.12,1610,A000091455,1448061,"[-75.6043865284, 6.21056733572]"
6,221550,2014-07-04,15:00:00,2014,7,11,Laureles Estadio,Las Acacias,Choque,Solo daños,Tramo de via,CL 33 CR 80,CL 033 080 000 00000,831466.19,1181887.52,1109,A000091403,1448095,"[-75.6002561334, 6.2387860571]"
7,57182,2014-07-04,15:30:00,2014,7,10,La Candelaria,Calle Nueva,Choque,Solo daños,Tramo de via,CL 37 CR 51,CL 037 051 000 00000,834460.82,1181840.32,1011,A000091503,1448080,"[-75.5732039534, 6.23843689866]"
8,177136,2014-07-04,15:40:00,2014,7,7,Robledo,San Germán,Choque,Solo daños,Tramo de via,CL 60 CR 74,CL 060 074 000 00000,832918.04,1184826.23,0703,A000091013,1447989,"[-75.587217576, 6.26538815378]"
9,221218,2014-07-04,16:10:00,2014,7,7,Robledo,Universidad Nacional,Choque,Solo daños,Tramo de via,CR 63 CL 58,CR 063 058 000 00000,834351.50,1184342.89,0701,A000091486,1448067,"[-75.5742557669, 6.26105610822]"


## 5) Exportación desde la base → CSV (streaming)

In [6]:
EXPORT_PATH = os.path.join(DB_DIR, 'export.csv')

cur2 = con.cursor()
cur2.execute('SELECT * FROM incidentes;')
colnames = [d[0] for d in cur2.description]

with open(EXPORT_PATH, 'w', newline='', encoding='utf-8') as f:
    import csv
    writer = csv.writer(f)
    writer.writerow(colnames)
    batch = cur2.fetchmany(50000)
    total_out = 0
    while batch:
        writer.writerows(batch)
        total_out += len(batch)
        print(f'Exportadas {total_out} filas...')
        batch = cur2.fetchmany(50000)

print('CSV exportado en:', EXPORT_PATH)


Exportadas 50000 filas...
Exportadas 100000 filas...
Exportadas 150000 filas...
Exportadas 200000 filas...
Exportadas 250000 filas...
Exportadas 270765 filas...
CSV exportado en: c:\Dev\proyecto_integrado_ea1\db\export.csv


## 6) Cierre
Base creada, datos cargados y CSV exportado.

**Archivos generados:** `db/proyecto.db` y `db/export.csv`.